# Fitting kinetic parameters to experimental data
Author: Björn Dahlgren.

Let us consider the reaction:

$$
Fe^{3+} + SCN^- \rightarrow FeSCN^{2+}
$$

the product is strongly coloured and we have experimental data of the absorbance as function of time after mixing for several replicates. The experiment was performed at 7 different temperatures and for one temperature, 7 different ionic strengths. For each set of conditions the experiment was re-run 7 times (replicates). In this notebook, we will determine the activation enthalpy and entropy through regressions analysis, we will also look at the ionic strength dependence.

In [ ]:
import numpy as np
import chempy
import chempy.equilibria
from chempy.units import default_units as u
from chempy.electrolytes import ionic_strength
from chempy._solution import QuantityDict
from chempy.printing import as_per_substance_html_table
print(chempy.__version__)

Experimental conditions, the two solutions which were mixed in 1:1 volume ration in a stopped flow apparatus:

In [ ]:
sol1 = QuantityDict(u.mM, {'SCN-': 3*u.mM, 'K+': 3*u.mM, 'Na+': 33*u.mM, 'H+': 50*u.mM, 'ClO4-': (33+50)*u.mM})
sol2 = QuantityDict(u.mM, {'Fe+3': 6*u.mM, 'H+': 50*u.mM, 'ClO4-': (3*6+50)*u.mM})

In [ ]:
sol = (sol1 + sol2)/2  # 1:1 volume ratio at mixing
sol.quantity_name = 'concentration'
sol

In [ ]:
eqsys = chempy.equilibria.EqSystem.from_string('Fe+3 + SCN- = FeSCN+2')
eqsys